# 3.3 Pre-entrenamiento 

En esta sección, se implementarán diversos AE con el fin de pre-entrenar redes profundas. De esta forma, se espera poder regularizar un modelo determinado, buscando evitar que este sufra de overfitting. 

**Importación de módulos necesarios**

In [30]:
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Input
from keras.models import Sequential, Model, load_model, save_model
from keras.utils import np_utils
from keras.optimizers import SGD
import numpy as np
import warnings

**3.3.0 Creación de conjuntos de datos a utilizar**

Análogamente a las secciones 3.1 y 3.2, se construyen los conjuntos de entrenamiento, validación y prueba.

In [13]:
# Se cargan conjutos de entrenamiento y de prueba
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Se normalizan conjuntos de datos en base a intensidad máxima de pixel
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Se transforman las imágenes de ambos conjuntos a vectores
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Se define conjunto de validación y se reestructura conjunto de entrenamiento
x_val = x_train[:5000, :]
x_train = x_train[5000:, :]
y_val = y_train[:5000]
y_train = y_train[5000:]

# Vectores y_train, y_val e y_test son transformados en matrices categóricas
y_train_ = np_utils.to_categorical(y_train)
y_val_ = np_utils.to_categorical(y_val)
y_test_ = np_utils.to_categorical(y_test)

**3.3.1 Definición de modelo a ser pre-entrenado**

En primer lugar, se diseña la red neuronal que será pre-entrenada. Está será una red FF de dos capas ocultas, las que poseerán 1000 neuronas, cada una. La función de activación de las neuronas será sigmoide. La capa de salida estará conformada por 10 neuronas (una por cada dígito), las que serán activadas por medio de la función softmax.

Para efectos comparativos, se entrenará la red sin pre-entrenamiento y se determinará el error de clasificación sobre el conjunto de pruebas. Este entrenamiento será llevado cabo a través del método de optimización *adam*, 50 epochs, batches de tamaño 25 y categorical crossentropy como función de pérdida. **Notar que a pesar de que se dió la instrucción de utilizar SGD, esto no ha sido posible, ya que al entrenar el modelo con este método, las tasas de error obtenidas (tanto de entrenamiento como de validación) eran cercanas al 91%**.

In [19]:
# Se define red vacía
model = Sequential()
# Se agregan capas ocultas
model.add(Dense(1000, activation='sigmoid', input_shape=(784,)))
model.add(Dense(1000, activation='sigmoid'))
# Se agrega capa de salida
model.add(Dense(10, activation='softmax'))
# Se definen parámetros de entrenamiento
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Se entrena modelo
model.fit(x_train, y_train_, epochs=50, batch_size=25, shuffle=True, validation_data=(x_val, y_val_))

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 41s - loss: 0.3219 - acc: 0.9007 - val_loss: 0.1339 - val_acc: 0.9584
Epoch 2/50
55000/55000 [==============================] - 41s - loss: 0.1259 - acc: 0.9615 - val_loss: 0.1091 - val_acc: 0.9682
Epoch 3/50
55000/55000 [==============================] - 41s - loss: 0.0790 - acc: 0.9755 - val_loss: 0.0898 - val_acc: 0.9728
Epoch 4/50
55000/55000 [==============================] - 42s - loss: 0.0542 - acc: 0.9827 - val_loss: 0.0759 - val_acc: 0.9776
Epoch 5/50
55000/55000 [==============================] - 42s - loss: 0.0388 - acc: 0.9874 - val_loss: 0.0703 - val_acc: 0.9802
Epoch 6/50
55000/55000 [==============================] - 43s - loss: 0.0297 - acc: 0.9900 - val_loss: 0.0924 - val_acc: 0.9744
Epoch 7/50
55000/55000 [==============================] - 43s - loss: 0.0229 - acc: 0.9923 - val_loss: 0.0767 - val_acc: 0.9814
Epoch 8/50
55000/55000 [==============================]

In [20]:
# Se determina error de clasificación sobre conjunto de pruebas
model_scores = model.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_scores[1]) * 100, '%'

 9760/10000 [============================>.] - ETA: 0s
Error de clasificacion sobre conjunto de pruebas:  1.79 %


Así, el entrenamiento finaliza con función de pérdida 0,0019 y error de clasificación de entrenamiento de un 0,04%. Además, el error de clasificación sobre el conjunto de pruebas es de un 1,79%.

**3.3.2 Pre-entrenando el modelo**

Con el fin de mejorar el error de clasificación, se pre-entrenará la red diseñada en la sección anterior. Como dicha red posee dos capas ocultas, se utilizarán dos autoencoders, cada uno encargado de pre-entrenar los pesos de una y sólo una de las capas. Para llevar a cabo esta tarea, se implementa la función *pre_training*, la que recibe el nombre de la función de activación a ser utilizada tanto en los autoencoders como en el modelo. Como resultado de este proceso, el modelo de la sección anterior será entrenado de tal manera que los pesos de cada capa oculta serán los mismos que pre-entrenó el autoencoder respectivo.

In [40]:
# pre_training permite el pre-entrenamiento de una red FF
def pre_training(encoder_activation, decoder_activation=None):
    
    if not decoder_activation:
        decoder_activation = encoder_activation
    
    # Se implementa primer autoencoder, encargado de pre-entrenar primera capa oculta
    # Se define input a ser recibido por autoencoder
    input_img1 = Input(shape=(784, ))
    # Se define capa encoder 
    encoded1 = Dense(1000, activation=encoder_activation)(input_img1)
    # Se define capa decoder
    decoded1 = Dense(784, activation=decoder_activation)(encoded1)
    # Se define autoencoder
    autoencoder1 = Model(input=input_img1, output=decoded1)
    # Se define encoder por separado
    encoder1 = Model(input_img1, output=encoded1)
    # Se definen parámetros de pre-entrenamiento
    autoencoder1.compile(optimizer='adam', loss='binary_crossentropy')
    # Se pre-entrena primer autoencoder
    autoencoder1.fit(x_train, x_train, epochs=50, batch_size=25, shuffle=True, verbose=0, validation_data=(x_val, x_val))
    encoded_input1 = Input(shape=(1000, ))
    # Se guardan autoencoder y encoder en archivos
    autoencoder1.save('autoencoder_1_' + encoder_activation + '.h5')
    encoder1.save('encoder_1_' + encoder_activation + '.h5')
    
    # Se implementa segundo autoencoder, encargado de pre-entrenar segunda capa oculta
    # Primero, se obtienen las salidas generadas por el encoder del primer autoencoder para cada conjunto de datos
    x_train_encoded1 = encoder1.predict(x_train)
    x_val_encoded1 = encoder1.predict(x_val)
    x_test_encoded1 = encoder1.predict(x_test)

    # Se define input a ser recibido por autoencoder
    input_img2 = Input(shape=(1000, ))
    # Se define capa encoder
    encoded2 = Dense(1000, activation=encoder_activation)(input_img2)
    # Se define capa decoder
    decoded2 = Dense(1000, activation=decoder_activation)(encoded2)
    # Se define autoencoder
    autoencoder2 = Model(input=input_img2, output=decoded2)
    # Se define encoder por separado
    encoder2 = Model(input=input_img2, output=encoded2)
    # Se especifican parámetros de optimización
    autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')
    # Se pre-entrena segundo autoencoder
    autoencoder2.fit(x_train_encoded1, x_train_encoded1, epochs=50, batch_size=25, shuffle=True, verbose=0,
                 validation_data=(x_val_encoded1, x_val_encoded1))
    encoded_input2 = Input(shape=(1000, ))
    autoencoder2.save('autoencoder_2_' + encoder_activation + '.h5')
    encoder2.save('encoder_2_' + encoder_activation + '.h5')

    # Finalmente, se entrena modelo a partir de pesos pre-entrenados
    model = Sequential()
    # Se agrega primera capa oculta
    model.add(Dense(1000, activation=encoder_activation, input_shape=(784, )))
    # Los pesos de la primera capa oculta son igualados a pesos pre-entrenados por primer autoencoder
    model.layers[-1].set_weights(autoencoder1.layers[1].get_weights())
    # Se agrega segunda capa oculta
    model.add(Dense(1000, activation=encoder_activation))
    # Los pesos de la segunda capa oculta son igualados a pesos pre-entrenados por segundo autoencoder
    model.layers[-1].set_weights(autoencoder2.layers[1].get_weights())
    # Se define capa de salida
    model.add(Dense(10, activation='softmax'))
    # Se especifican parámetros de optimización
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Se entrena modeo
    model.fit(x_train, y_train_, epochs=20, batch_size=25, shuffle=True, verbose=1 , validation_data=(x_val, y_val_))
    # Se guarda modelo entrenado en archivo
    model.save('Net-784x1000x1000x10-finetunned_' + encoder_activation + '.h5')

Así, se ejecuta el proceso descrito sobre el modelo de la sección anterior utilizando la función de activación sigmoide tanto en los autoencoders como en las capas ocultas del modelo.

In [32]:
warnings.filterwarnings('ignore')

pre_training('sigmoid')
model_sigmoid = load_model('Net-784x1000x1000x10-finetunned_sigmoid.h5')
model_sigmoid_scores = model_sigmoid.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_sigmoid_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 41s - loss: 0.2367 - acc: 0.9277 - val_loss: 0.1243 - val_acc: 0.9630
Epoch 2/20
55000/55000 [==============================] - 41s - loss: 0.0889 - acc: 0.9722 - val_loss: 0.0866 - val_acc: 0.9728
Epoch 3/20
55000/55000 [==============================] - 41s - loss: 0.0537 - acc: 0.9831 - val_loss: 0.0817 - val_acc: 0.9742
Epoch 4/20
55000/55000 [==============================] - 41s - loss: 0.0360 - acc: 0.9883 - val_loss: 0.0769 - val_acc: 0.9796
Epoch 5/20
55000/55000 [==============================] - 41s - loss: 0.0243 - acc: 0.9920 - val_loss: 0.0764 - val_acc: 0.9802
Epoch 6/20
55000/55000 [==============================] - 41s - loss: 0.0198 - acc: 0.9935 - val_loss: 0.0696 - val_acc: 0.9816
Epoch 7/20
55000/55000 [==============================] - 41s - loss: 0.0156 - acc: 0.9944 - val_loss: 0.0691 - val_acc: 0.9826
Epoch 8/20
55000/55000 [==============================]

Se ve entonces que ...

**3.3.3 Pre-entrenamiento vía *denoising autoencoder***

In [34]:
warnings.filterwarnings('ignore')

pre_training('tanh')
model_tanh = load_model('Net-784x1000x1000x10-finetunned_tanh.h5')
model_tanh_scores = model_tanh.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_tanh_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 38s - loss: 0.2519 - acc: 0.9237 - val_loss: 0.1382 - val_acc: 0.9596
Epoch 2/20
55000/55000 [==============================] - 38s - loss: 0.1325 - acc: 0.9594 - val_loss: 0.1143 - val_acc: 0.9668
Epoch 3/20
55000/55000 [==============================] - 38s - loss: 0.1108 - acc: 0.9659 - val_loss: 0.1159 - val_acc: 0.9686
Epoch 4/20
55000/55000 [==============================] - 39s - loss: 0.0943 - acc: 0.9705 - val_loss: 0.1024 - val_acc: 0.9726
Epoch 5/20
55000/55000 [==============================] - 39s - loss: 0.0826 - acc: 0.9747 - val_loss: 0.1101 - val_acc: 0.9654
Epoch 6/20
55000/55000 [==============================] - 39s - loss: 0.0747 - acc: 0.9769 - val_loss: 0.1044 - val_acc: 0.9716
Epoch 7/20
55000/55000 [==============================] - 40s - loss: 0.0712 - acc: 0.9767 - val_loss: 0.1284 - val_acc: 0.9684
Epoch 8/20
55000/55000 [==============================]

Recordando que es una mala práctica...

In [41]:
warnings.filterwarnings('ignore')

pre_training('relu', 'sigmoid')
model_relu = load_model('Net-784x1000x1000x10-finetunned_relu.h5')
model_relu_scores = model_relu.evaluate(x_test, y_test_)
print '\nError de clasificacion sobre conjunto de pruebas: ', (1 - model_relu_scores[1]) * 100, '%'

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 38s - loss: 0.2207 - acc: 0.9394 - val_loss: 0.0947 - val_acc: 0.9704
Epoch 2/20
55000/55000 [==============================] - 39s - loss: 0.0782 - acc: 0.9758 - val_loss: 0.0998 - val_acc: 0.9744
Epoch 3/20
55000/55000 [==============================] - 39s - loss: 0.0563 - acc: 0.9825 - val_loss: 0.1028 - val_acc: 0.9758
Epoch 4/20
55000/55000 [==============================] - 40s - loss: 0.0430 - acc: 0.9866 - val_loss: 0.0860 - val_acc: 0.9788
Epoch 5/20
55000/55000 [==============================] - 40s - loss: 0.0333 - acc: 0.9895 - val_loss: 0.0985 - val_acc: 0.9796
Epoch 6/20
55000/55000 [==============================] - 40s - loss: 0.0311 - acc: 0.9909 - val_loss: 0.1206 - val_acc: 0.9766
Epoch 7/20
55000/55000 [==============================] - 40s - loss: 0.0324 - acc: 0.9911 - val_loss: 0.1204 - val_acc: 0.9776
Epoch 8/20
55000/55000 [==============================]